This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## Neural style transfer

### The content loss

### The style loss

### Neural style transfer in Keras

**Getting the style and content images**

In [ ]:
from tensorflow import keras # importing keras from tensorflow

base_image_path = keras.utils.get_file( # getting the base image
    "sf.jpg", origin="https://img-datasets.s3.amazonaws.com/sf.jpg") # origin of the image
style_reference_image_path = keras.utils.get_file( # getting the style reference image
    "starry_night.jpg", origin="https://img-datasets.s3.amazonaws.com/starry_night.jpg") # origin of the image
 
original_width, original_height = keras.utils.load_img(base_image_path).size # loading the base image and getting the size of the image 
img_height = 400 # setting the image height to 400
img_width = round(original_width * img_height / original_height) # setting the image width to the original width times the image height divided by the original height

**Auxiliary functions**

In [ ]:
import numpy as np # importing numpy

def preprocess_image(image_path): # defining the preprocess image function
    img = keras.utils.load_img( # loading the image
        image_path, target_size=(img_height, img_width)) # setting the target size of the image
    img = keras.utils.img_to_array(img) # converting the image to an array
    img = np.expand_dims(img, axis=0) # expanding the dimensions of the image array by adding an axis at the 0th position 
    img = keras.applications.vgg19.preprocess_input(img) # preprocessing the image
    return img # returning the image

def deprocess_image(img): # defining the deprocess image function
    img = img.reshape((img_height, img_width, 3)) # reshaping the image
    img[:, :, 0] += 103.939 # adding 103.939 to the first channel of the image
    img[:, :, 1] += 116.779 # adding 116.779 to the second channel of the image
    img[:, :, 2] += 123.68 # adding 123.68 to the third channel of the image
    img = img[:, :, ::-1] # reversing the order of the channels
    img = np.clip(img, 0, 255).astype("uint8") # clipping the image and converting it to an unsigned integer
    return img # returning the image

**Using a pretrained VGG19 model to create a feature extractor**

In [ ]:
model = keras.applications.vgg19.VGG19(weights="imagenet", include_top=False) # loading the VGG19 model with the imagenet weights and excluding the top layer of the model (the classification layer)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers]) # creating a dictionary of the model layers and their outputs 
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict) # creating a model that extracts features from the model layers

**Content loss**

In [ ]:
def content_loss(base_img, combination_img): # defining the content loss function with the base image and the combination image as parameters
    return tf.reduce_sum(tf.square(combination_img - base_img)) # returning the sum of the squared difference between the combination image and the base image

**Style loss**

In [ ]:
def gram_matrix(x): # defining the gram matrix function with x as a parameter
    x = tf.transpose(x, (2, 0, 1)) # transposing the x tensor to have the channels first and the height and width second 
    features = tf.reshape(x, (tf.shape(x)[0], -1)) # reshaping the x tensor to have the first dimension as the number of channels and the second dimension as the product of the height and width
    gram = tf.matmul(features, tf.transpose(features)) # multiplying the features tensor by its transpose
    return gram # returning the gram matrix

def style_loss(style_img, combination_img): # defining the style loss function with the style image and the combination image as parameters
    S = gram_matrix(style_img) # calculating the gram matrix of the style image
    C = gram_matrix(combination_img) # calculating the gram matrix of the combination image
    channels = 3 # setting the number of channels to 3
    size = img_height * img_width # setting the size to the product of the image height and width
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2)) # returning the sum of the squared difference between the gram matrices divided by 4 times the square of the number of channels times the square of the size

**Total variation loss**

In [ ]:
def total_variation_loss(x): # defining the total variation loss function with x as a parameter
    a = tf.square( # calculating the square of the difference between the first and second rows of the x tensor
        x[:, : img_height - 1, : img_width - 1, :] - x[:, 1:, : img_width - 1, :] # first row of the x tensor minus the second row of the x tensor 
    )
    b = tf.square( # calculating the square of the difference between the first and second columns of the x tensor
        x[:, : img_height - 1, : img_width - 1, :] - x[:, : img_height - 1, 1:, :] # first column of the x tensor minus the second column of the x tensor
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25)) # returning the sum of the power of the sum of a and b to the 1.25th power

**Defining the final loss that you'll minimize**

In [ ]:
style_layer_names = [ # defining the style layer names
    "block1_conv1", # first convolutional layer in block 1
    "block2_conv1", # first convolutional layer in block 2
    "block3_conv1", # first convolutional layer in block 3
    "block4_conv1", # first convolutional layer in block 4
    "block5_conv1", # first convolutional layer in block 5
]
content_layer_name = "block5_conv2" # defining the content layer name
total_variation_weight = 1e-6 # setting the total variation weight to 1e-6 (0.000001) 
style_weight = 1e-6 # setting the style weight to 1e-6 (0.000001) 
content_weight = 2.5e-8 # setting the content weight to 2.5e-8 (0.000000025)

def compute_loss(combination_image, base_image, style_reference_image): # defining the compute loss function with the combination image, base image, and style reference image as parameters
    input_tensor = tf.concat( # concatenating the base image, style reference image, and combination image along the first axis
        [base_image, style_reference_image, combination_image], axis=0 # base image, style reference image, and combination image along the first axis 
    )
    features = feature_extractor(input_tensor) # extracting features from the input tensor
    loss = tf.zeros(shape=()) # setting the loss to a tensor with a shape of ()
    layer_features = features[content_layer_name] # getting the features of the content layer
    base_image_features = layer_features[0, :, :, :] # getting the features of the base image
    combination_features = layer_features[2, :, :, :] # getting the features of the combination image
    loss = loss + content_weight * content_loss( # adding the content weight times the content loss to the loss
        base_image_features, combination_features # base image features and combination features as parameters
    )
    for layer_name in style_layer_names: # iterating over the style layer names
        layer_features = features[layer_name] # getting the features of the layer
        style_reference_features = layer_features[1, :, :, :] # getting the features of the style reference image
        combination_features = layer_features[2, :, :, :] # getting the features of the combination image
        style_loss_value = style_loss( # calculating the style loss value
          style_reference_features, combination_features) # style reference features and combination features as parameters
        loss += (style_weight / len(style_layer_names)) * style_loss_value # adding the style weight divided by the length of the style layer names times the style loss value to the loss
 
    loss += total_variation_weight * total_variation_loss(combination_image) # adding the total variation weight times the total variation loss to the loss
    return loss # returning the loss

**Setting up the gradient-descent process**

In [ ]:
import tensorflow as tf # importing tensorflow
 
@tf.function # defining the compute loss and grads function as a TensorFlow function 
def compute_loss_and_grads(combination_image, base_image, style_reference_image): # defining the compute loss and grads function with the combination image, base image, and style reference image as parameters
    with tf.GradientTape() as tape: # creating a gradient tape
        loss = compute_loss(combination_image, base_image, style_reference_image) # calculating the loss
    grads = tape.gradient(loss, combination_image) # calculating the gradients
    return loss, grads # returning the loss and gradients

optimizer = keras.optimizers.SGD( # defining the optimizer as stochastic gradient descent
    keras.optimizers.schedules.ExponentialDecay( # using an exponential decay learning rate schedule
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96 # initial learning rate of 100.0, decay steps of 100, and decay rate of 0.96
    )
)

base_image = preprocess_image(base_image_path) # preprocessing the base image
style_reference_image = preprocess_image(style_reference_image_path) # preprocessing the style reference image
combination_image = tf.Variable(preprocess_image(base_image_path)) # creating a TensorFlow variable for the combination image

iterations = 4000 # setting the number of iterations to 4000
for i in range(1, iterations + 1): # iterating over the range of iterations
    loss, grads = compute_loss_and_grads( # calculating the loss and gradients
        combination_image, base_image, style_reference_image # combination image, base image, and style reference image as parameters
    )
    optimizer.apply_gradients([(grads, combination_image)]) # applying the gradients to the combination image
    if i % 100 == 0: # if the iteration is divisible by 100
        print(f"Iteration {i}: loss={loss:.2f}") # print the iteration and loss
        img = deprocess_image(combination_image.numpy()) # deprocess the combination image
        fname = f"combination_image_at_iteration_{i}.png" # setting the file name
        keras.utils.save_img(fname, img) # saving the image

### Wrapping up